In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
%matplotlib inline

In [2]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        # 3x3 Convolution: 9x9 --> 5x5x(30 features)
        self.conv = nn.Conv2d(1,30,3)
        self.pool = nn.MaxPool2d((2,2))
        self.l1 = nn.Linear(30*3*3,200)
        self.l2 = nn.Linear(200,120)
        self.l3 = nn.Linear(120, 81)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv(x)))
        x= x.view(-1, self.num_flat_features(x))
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.l3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    

In [3]:
import os
import re
import go
from parse_sgf import get_moves

def randReflect(board, move):
    if np.random.randint(2):
        board = board.T
        (x, y) = move
        move = (y,x)
    return board, move

class NinebyNineGames(Dataset):
    def __init__(self, root_dir, transform = None):
        '''Read all the .sgf files from root_dir'''
        self.transform = transform
        self.root_dir = root_dir
        self.sgf_files = [entry for entry in os.scandir(self.root_dir) if entry.path.endswith(".sgf")]
        self.games = [get_moves(sgf) for sgf in self.sgf_files]
            
    def __len__(self):
        return sum( len(game) for game in self.games)
    
    def __getitem__(self, idx):
        '''return the position as a (9,9) ndarray
        and next move as a squashed coordinate 0 - 80'''
        g_idx = 0
        count = 0
        while(idx > count + len(self.games[g_idx]) -1 ) :
            count += len(self.games[g_idx])
            g_idx += 1
        g = go.Game( moves = self.games[g_idx])
        
        for _ in range(idx-count):
                g.play_move()
        board, move =  np.array(g.get_board()).reshape(9,9) , g.moves[idx - count]
        if self.transform:
            board , move = self.transform(board, move)
        return torch.Tensor(board).unsqueeze(0) , go.squash(move)

In [8]:
dir = r"/home/jupyter/BokeGo/data"
data = NinebyNineGames(dir)
dataloader = DataLoader(data, batch_size = 32, shuffle = True)

pi = Policy()
err = nn.CrossEntropyLoss()
optimizer = optim.Adam(pi.parameters(), lr = 0.001)

In [12]:
#GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
pi.to(device)

epochs = 2
for _ in range(epochs):
    running_loss = 0.0
    for i, data in tqdm(enumerate(dataloader,0)):
        inputs, moves = data
        inputs, moves = inputs.to(device), moves.to(device)
        
        optimizer.zero_grad()
        outputs = pi(inputs)
        
        #backprop
        loss = err(outputs, moves)
        loss.backward()
        optimizer.step()
    
        running_loss += loss.item()
        if i % 10000 == 9999 :
            print(f"Epoch: {epoch+1}, Loss: {running_loss:.3f}")
            running_loss = 0.0



0it [00:00, ?it/s]
1it [00:00,  7.75it/s]

cuda:0



3it [00:00,  8.58it/s]
5it [00:00,  9.21it/s]
7it [00:00, 10.08it/s]
9it [00:00, 10.72it/s]
11it [00:00, 11.20it/s]
13it [00:01, 11.52it/s]
15it [00:01, 11.81it/s]
17it [00:01, 12.01it/s]
19it [00:01, 12.31it/s]
21it [00:01, 11.76it/s]
23it [00:01, 11.99it/s]
25it [00:02, 11.56it/s]
27it [00:02, 11.84it/s]
29it [00:02, 11.96it/s]
31it [00:02, 12.10it/s]
33it [00:02, 11.81it/s]
35it [00:02, 11.53it/s]
37it [00:03, 11.26it/s]
39it [00:03, 11.62it/s]
41it [00:03, 11.87it/s]
43it [00:03, 11.97it/s]
45it [00:03, 11.98it/s]
47it [00:03, 11.78it/s]
49it [00:04, 11.78it/s]
51it [00:04, 12.40it/s]
53it [00:04, 12.13it/s]
55it [00:04, 12.39it/s]
57it [00:04, 12.32it/s]
59it [00:04, 12.72it/s]
61it [00:05, 12.97it/s]
63it [00:05, 12.31it/s]
65it [00:05, 11.96it/s]
67it [00:05, 12.17it/s]
69it [00:05, 12.06it/s]
71it [00:05, 11.94it/s]
73it [00:06, 12.14it/s]
75it [00:06, 11.99it/s]
77it [00:06, 12.06it/s]
79it [00:06, 12.21it/s]
81it [00:06, 11.81it/s]
83it [00:06, 12.04it/s]
85it [00:07, 11.74i

1169it [01:35, 12.59it/s]
1171it [01:35, 13.14it/s]
1173it [01:35, 12.90it/s]
1175it [01:35, 12.60it/s]
1177it [01:35, 12.71it/s]
1179it [01:35, 12.34it/s]
1181it [01:36, 12.26it/s]
1183it [01:36, 12.55it/s]
1185it [01:36, 12.57it/s]
1187it [01:36, 12.95it/s]
1189it [01:36, 12.63it/s]
1191it [01:36, 12.80it/s]
1193it [01:36, 13.21it/s]
1195it [01:37, 12.91it/s]
1197it [01:37, 13.20it/s]
1199it [01:37, 13.09it/s]
1201it [01:37, 12.56it/s]
1203it [01:37, 12.38it/s]
1205it [01:37, 12.18it/s]
1207it [01:38, 12.03it/s]
1209it [01:38, 11.81it/s]
1211it [01:38, 11.75it/s]
1213it [01:38, 12.12it/s]
1215it [01:38, 12.13it/s]
1217it [01:38, 12.03it/s]
1219it [01:39, 12.49it/s]
1221it [01:39, 12.60it/s]
1223it [01:39, 12.10it/s]
1225it [01:39, 12.39it/s]
1227it [01:39, 12.41it/s]
1229it [01:39, 13.00it/s]
1231it [01:39, 13.01it/s]
1233it [01:40, 12.98it/s]
1235it [01:40, 12.73it/s]
1237it [01:40, 12.32it/s]
1239it [01:40, 12.04it/s]
1241it [01:40, 12.34it/s]
1243it [01:40, 12.49it/s]
1245it [01:4

2297it [03:06, 11.98it/s]
2299it [03:06, 12.04it/s]
2301it [03:06, 12.34it/s]
2303it [03:06, 12.31it/s]
2305it [03:06, 12.21it/s]
2307it [03:07, 12.45it/s]
2309it [03:07, 12.16it/s]
2311it [03:07, 12.42it/s]
2313it [03:07, 12.55it/s]
2315it [03:07, 12.86it/s]
2317it [03:07, 13.01it/s]
2319it [03:08, 12.73it/s]
2321it [03:08, 12.65it/s]
2323it [03:08, 12.49it/s]
2325it [03:08, 12.95it/s]
2327it [03:08, 12.56it/s]
2329it [03:08, 11.98it/s]
2331it [03:09, 12.08it/s]
2333it [03:09, 11.90it/s]
2335it [03:09, 12.17it/s]
2337it [03:09, 12.34it/s]
2339it [03:09, 12.83it/s]
2341it [03:09, 13.11it/s]
2343it [03:09, 12.95it/s]
2345it [03:10, 12.83it/s]
2347it [03:10, 12.41it/s]
2349it [03:10, 12.55it/s]
2351it [03:10, 12.65it/s]
2353it [03:10, 12.64it/s]
2355it [03:10, 12.69it/s]
2357it [03:11, 12.51it/s]
2359it [03:11, 12.64it/s]
2361it [03:11, 12.32it/s]
2363it [03:11, 12.56it/s]
2365it [03:11, 12.22it/s]
2367it [03:11, 11.88it/s]
2369it [03:12, 12.33it/s]
2371it [03:12, 12.43it/s]
2373it [03:1

...

18739it [25:47, 12.04it/s]
18741it [25:47, 12.10it/s]
18743it [25:47, 11.95it/s]
18745it [25:47, 12.26it/s]
18747it [25:47, 12.03it/s]
18749it [25:47, 12.28it/s]
18751it [25:48, 12.05it/s]
18753it [25:48, 12.04it/s]
18755it [25:48, 11.56it/s]
18757it [25:48, 11.64it/s]
18759it [25:48, 12.04it/s]
18761it [25:48, 11.86it/s]
18763it [25:49, 12.14it/s]
18765it [25:49, 12.12it/s]
18767it [25:49, 12.13it/s]
18769it [25:49, 12.04it/s]
18771it [25:49, 12.16it/s]
18773it [25:49, 12.34it/s]
18775it [25:49, 12.45it/s]
18777it [25:50, 12.40it/s]
18779it [25:50, 12.40it/s]
18781it [25:50, 12.21it/s]
18783it [25:50, 12.72it/s]
18785it [25:50, 12.54it/s]
18787it [25:50, 12.52it/s]
18789it [25:51, 12.39it/s]
18791it [25:51, 12.47it/s]
18793it [25:51, 12.34it/s]
18795it [25:51, 12.06it/s]
18797it [25:51, 12.33it/s]
18799it [25:51, 12.23it/s]
18801it [25:52, 12.30it/s]
18803it [25:52, 12.68it/s]
18805it [25:52, 12.73it/s]
18807it [25:52, 12.48it/s]
18809it [25:52, 12.12it/s]
18811it [25:52, 12.45it/s]
1

19831it [27:17, 11.55it/s]
19833it [27:17, 11.54it/s]
19835it [27:17, 11.62it/s]
19837it [27:17, 11.61it/s]
19839it [27:18, 11.61it/s]
19841it [27:18, 11.61it/s]
19843it [27:18, 11.51it/s]
19845it [27:18, 12.15it/s]
19847it [27:18, 12.21it/s]
19849it [27:18, 12.46it/s]
19851it [27:19, 12.25it/s]
19853it [27:19, 11.85it/s]
19855it [27:19, 11.43it/s]
19857it [27:19, 11.17it/s]
19859it [27:19, 11.47it/s]
19861it [27:19, 11.59it/s]
19863it [27:20, 11.44it/s]
19865it [27:20, 11.47it/s]
19867it [27:20, 11.91it/s]
19869it [27:20, 12.50it/s]
19871it [27:20, 12.14it/s]
19873it [27:20, 12.27it/s]
19875it [27:21, 11.65it/s]
19877it [27:21, 11.38it/s]
19879it [27:21, 11.76it/s]
19881it [27:21, 12.36it/s]
19883it [27:21, 12.45it/s]
19885it [27:21, 12.07it/s]
19887it [27:22, 11.94it/s]
19889it [27:22, 12.51it/s]
19891it [27:22, 12.96it/s]
19893it [27:22, 12.36it/s]
19895it [27:22, 12.34it/s]
19897it [27:22, 12.44it/s]
19899it [27:23, 12.05it/s]
19901it [27:23, 12.17it/s]
19903it [27:23, 12.19it/s]
1

Epoch: 1, Loss: 38215.961



20003it [27:31, 12.14it/s]
20005it [27:31, 12.25it/s]
20007it [27:31, 12.51it/s]
20009it [27:32, 12.08it/s]
20011it [27:32, 11.77it/s]
20013it [27:32, 12.09it/s]
20015it [27:32, 12.27it/s]
20017it [27:32, 12.15it/s]
20019it [27:32, 12.01it/s]
20021it [27:33, 12.31it/s]
20023it [27:33, 11.87it/s]
20025it [27:33, 11.84it/s]
20027it [27:33, 11.96it/s]
20029it [27:33, 11.92it/s]
20031it [27:33, 11.73it/s]
20033it [27:34, 11.96it/s]
20035it [27:34, 11.99it/s]
20037it [27:34, 11.97it/s]
20039it [27:34, 12.23it/s]
20041it [27:34, 12.37it/s]
20043it [27:34, 12.47it/s]
20045it [27:35, 12.41it/s]
20047it [27:35, 12.16it/s]
20049it [27:35, 12.38it/s]
20051it [27:35, 12.32it/s]
20053it [27:35, 12.40it/s]
20055it [27:35, 12.29it/s]
20057it [27:36, 12.71it/s]
20059it [27:36, 12.77it/s]
20061it [27:36, 12.75it/s]
20063it [27:36, 13.04it/s]
20065it [27:36, 12.91it/s]
20067it [27:36, 12.51it/s]
20069it [27:37, 12.17it/s]
20071it [27:37, 12.50it/s]
20073it [27:37, 12.19it/s]
20075it [27:37, 12.47it/s]


21095it [29:02, 12.09it/s]
21097it [29:03, 12.63it/s]
21099it [29:03, 12.72it/s]
21101it [29:03, 12.50it/s]
21103it [29:03, 12.33it/s]
21105it [29:03, 12.37it/s]
21107it [29:03, 12.24it/s]
21109it [29:04, 12.33it/s]
21111it [29:04, 12.35it/s]
21113it [29:04, 12.50it/s]
21115it [29:04, 12.46it/s]
21117it [29:04, 12.53it/s]
21119it [29:04, 13.10it/s]
21121it [29:04, 12.52it/s]
21123it [29:05, 12.62it/s]
21125it [29:05, 12.61it/s]
21127it [29:05, 12.68it/s]
21129it [29:05, 12.77it/s]
21131it [29:05, 12.40it/s]
21133it [29:05, 12.51it/s]
21135it [29:06, 12.31it/s]
21137it [29:06, 11.87it/s]
21139it [29:06, 11.22it/s]
21141it [29:06, 11.71it/s]
21143it [29:06, 11.71it/s]
21145it [29:06, 11.99it/s]
21147it [29:07, 11.99it/s]
21149it [29:07, 11.86it/s]
21151it [29:07, 11.71it/s]
21153it [29:07, 12.07it/s]
21155it [29:07, 12.22it/s]
21157it [29:07, 12.31it/s]
21159it [29:08, 11.76it/s]
21161it [29:08, 11.82it/s]
21163it [29:08, 12.01it/s]
21165it [29:08, 12.46it/s]
21167it [29:08, 12.50it/s]
2

In [15]:
torch.save(pi.state_dict(), r"/home/jupyter/BokeGo/policy_train_1.pt")